In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from matplotlib.patches import Rectangle, Circle
import matplotlib.patches as mpatches
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# USER INPUT PARAMETERS - CUSTOMIZE YOUR SIMULATION HERE
# ============================================================================
print("\n" + "=" * 60)
print("EHD PUMP PARAMETER CONFIGURATION")
print("=" * 60)

# Get user input for key parameters
use_default = input("Use default parameters? (y/n): ").lower().strip()

if use_default == 'n':
    print("\nEnter your custom parameters:")
    V_0 = float(input("  Applied voltage V₀ (kV) [default=22]: ") or 22) * 1e3
    rho_0 = float(input("  Charge density ρ₀ (mC/m³) [default=25]: ") or 25) * 1e-3
    d = float(input("  Electrode separation d (cm) [default=1]: ") or 1) * 0.01
    epsilon_r = float(input("  Relative permittivity εᵣ [default=80]: ") or 80)
    base_velocity_input = float(input("  Base particle velocity (mm/s) [default=1.2]: ") or 1.2)
    base_velocity = base_velocity_input * 1e-3
    n_particles = int(input("  Number of particles [default=150]: ") or 150)
else:
    # Default parameters
    V_0 = 22e3     # Voltage in V
    rho_0 = 25e-3  # Charge density in C/m³
    d = 0.01       # Distance between electrodes in meters (1 cm)
    epsilon_r = 80 # Relative permittivity (water)
    base_velocity = 0.0012  # m/s
    n_particles = 150

# Fixed physical constants
epsilon_0 = 8.854e-12  # Permittivity of free space
epsilon = epsilon_0 * epsilon_r

# Derived parameters
E_field = V_0 / d  # Electric field strength
pressure_gradient = rho_0 * E_field  # Pressure gradient
total_pressure = pressure_gradient * d  # Total pressure difference

print("=" * 60)
print("ELECTROHYDRODYNAMIC (EHD) PUMP ANALYSIS")
print("=" * 60)
print(f"Configured Parameters:")
print(f"  Charge density (ρ₀): {rho_0*1000:.1f} mC/m³")
print(f"  Applied voltage (V₀): {V_0/1000:.1f} kV")
print(f"  Electrode separation (d): {d*100:.1f} cm")
print(f"  Relative permittivity (εᵣ): {epsilon_r:.1f}")
print(f"  Permittivity (ε): {epsilon:.3e} F/m")
print(f"  Particle velocity: {base_velocity*1000:.2f} mm/s")
print(f"  Number of particles: {n_particles}")
print(f"\nCalculated Results:")
print(f"  Electric field (E): {E_field:.2e} V/m = {E_field/1e6:.2f} MV/m")
print(f"  Pressure gradient (dP/dz): {pressure_gradient:.2e} Pa/m")
print(f"  Total pressure (ΔP): {total_pressure:.2f} Pa")
print("=" * 60)

# Create spatial grid for potential distribution
z = np.linspace(0, d, 100)

# Solving Poisson's equation: d²V/dz² = -ρ₀/ε
# Boundary conditions: V(0) = V₀, V(d) = 0
# Solution: V(z) = -ρ₀z²/(2ε) + Az + B

# Apply boundary conditions to find A and B
B = V_0
A = (rho_0 * d / (2 * epsilon) - V_0) / d

# Potential distribution
V = -(rho_0 * z**2) / (2 * epsilon) + A * z + B

# Electric field E = -dV/dz
E = -(-(rho_0 * z) / epsilon + A)

# Pressure distribution P = ∫ρ₀E dz
P = rho_0 * E_field * z

# Create comprehensive plots
fig = plt.figure(figsize=(16, 10))

# 1. Potential Distribution
ax1 = plt.subplot(2, 3, 1)
ax1.plot(z * 100, V / 1000, 'b-', linewidth=2.5)
ax1.set_xlabel('Position z (cm)', fontsize=11, fontweight='bold')
ax1.set_ylabel('Potential V (kV)', fontsize=11, fontweight='bold')
ax1.set_title('Potential Distribution in EHD Pump', fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.axhline(y=0, color='r', linestyle='--', alpha=0.5, label='Ground')
ax1.axhline(y=V_0/1000, color='g', linestyle='--', alpha=0.5, label=f'V₀ = {V_0/1000} kV')
ax1.legend()

# 2. Electric Field Distribution
ax2 = plt.subplot(2, 3, 2)
ax2.plot(z * 100, E / 1e6, 'r-', linewidth=2.5)
ax2.set_xlabel('Position z (cm)', fontsize=11, fontweight='bold')
ax2.set_ylabel('Electric Field E (MV/m)', fontsize=11, fontweight='bold')
ax2.set_title('Electric Field Distribution', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.axhline(y=E_field/1e6, color='b', linestyle='--', alpha=0.5, label=f'E₀ = {E_field/1e6:.2f} MV/m')
ax2.legend()

# 3. Pressure Distribution
ax3 = plt.subplot(2, 3, 3)
ax3.plot(z * 100, P, 'g-', linewidth=2.5)
ax3.set_xlabel('Position z (cm)', fontsize=11, fontweight='bold')
ax3.set_ylabel('Pressure P (Pa)', fontsize=11, fontweight='bold')
ax3.set_title('Pressure Distribution', fontsize=12, fontweight='bold')
ax3.grid(True, alpha=0.3)
ax3.fill_between(z * 100, 0, P, alpha=0.3, color='green')

# 4. Schematic diagram
ax4 = plt.subplot(2, 3, 4)
ax4.set_xlim(-0.2, 1.2)
ax4.set_ylim(-0.2, 1.2)
ax4.set_aspect('equal')

# Draw electrodes
electrode_left = Rectangle((-0.1, 0.2), 0.1, 0.6, color='red', alpha=0.7)
electrode_right = Rectangle((1.0, 0.2), 0.1, 0.6, color='blue', alpha=0.7)
ax4.add_patch(electrode_left)
ax4.add_patch(electrode_right)

# Draw fluid region
fluid_region = Rectangle((0, 0.2), 1.0, 0.6, color='cyan', alpha=0.2)
ax4.add_patch(fluid_region)

# Add labels
ax4.text(-0.05, 0.9, '+V₀', fontsize=14, fontweight='bold', ha='center', color='red')
ax4.text(1.05, 0.9, '0V', fontsize=14, fontweight='bold', ha='center', color='blue')
ax4.text(0.5, 0.5, 'Charged Fluid\nρ₀ = 25 mC/m³', fontsize=11, ha='center', 
         bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# Draw arrows for flow direction
for i in range(5):
    x_pos = 0.15 + i * 0.15
    ax4.arrow(x_pos, 0.5, 0.1, 0, head_width=0.08, head_length=0.05, 
              fc='black', ec='black', linewidth=2)

ax4.text(0.5, 0.05, 'Flow Direction →', fontsize=12, ha='center', fontweight='bold')
ax4.set_title('EHD Pump Schematic', fontsize=12, fontweight='bold')
ax4.axis('off')

# 5. Charge density and force distribution
ax5 = plt.subplot(2, 3, 5)
force_density = rho_0 * E
ax5.plot(z * 100, np.ones_like(z) * rho_0 * 1000, 'purple', linewidth=2.5, label='Charge Density')
ax5.set_xlabel('Position z (cm)', fontsize=11, fontweight='bold')
ax5.set_ylabel('ρ₀ (mC/m³)', fontsize=11, fontweight='bold', color='purple')
ax5.tick_params(axis='y', labelcolor='purple')
ax5.set_title('Charge Density (Uniform)', fontsize=12, fontweight='bold')
ax5.grid(True, alpha=0.3)
ax5.legend(loc='upper right')

ax5_twin = ax5.twinx()
ax5_twin.plot(z * 100, force_density, 'orange', linewidth=2.5, label='Force Density')
ax5_twin.set_ylabel('Force Density ρ₀E (N/m³)', fontsize=11, fontweight='bold', color='orange')
ax5_twin.tick_params(axis='y', labelcolor='orange')
ax5_twin.legend(loc='upper left')

# 6. Energy analysis
ax6 = plt.subplot(2, 3, 6)
energy_density = 0.5 * epsilon * E**2
ax6.plot(z * 100, energy_density, 'magenta', linewidth=2.5)
ax6.set_xlabel('Position z (cm)', fontsize=11, fontweight='bold')
ax6.set_ylabel('Energy Density (J/m³)', fontsize=11, fontweight='bold')
ax6.set_title('Electric Energy Density Distribution', fontsize=12, fontweight='bold')
ax6.grid(True, alpha=0.3)
ax6.fill_between(z * 100, 0, energy_density, alpha=0.3, color='magenta')

plt.tight_layout()
plt.savefig('ehd_pump_analysis.png', dpi=300, bbox_inches='tight')
print("\n✓ Static analysis plots saved as 'ehd_pump_analysis.png'")
plt.show()

# ============================================================================
# ANIMATION: Electron/Charge Movement in 2D Cylindrical EHD Pump
# ============================================================================

print("\nGenerating animation... This may take a minute...")

# Animation parameters
cylinder_radius = 0.004  # 4 mm radius
cylinder_length = d
fps = 30
duration = 8  # seconds
n_frames = fps * duration

# Initialize particle positions
np.random.seed(42)
particle_z = np.random.uniform(0, cylinder_length, n_particles)
particle_r = np.random.uniform(-cylinder_radius, cylinder_radius, n_particles)
particle_theta = np.random.uniform(0, 2*np.pi, n_particles)

# Particle velocities (based on electric field strength)
# Higher electric field = faster particle movement
velocity_scale = E_field / 2.2e6  # Normalized to default E-field
particle_vz = np.ones(n_particles) * base_velocity * velocity_scale

# Create animation figure
fig_anim = plt.figure(figsize=(16, 8))
fig_anim.patch.set_facecolor('white')

# Top view (x-z plane)
ax_top = plt.subplot(1, 2, 1)
ax_top.set_xlim(-0.001, cylinder_length + 0.001)
ax_top.set_ylim(-cylinder_radius * 1.2, cylinder_radius * 1.2)
ax_top.set_xlabel('Axial Position z (m)', fontsize=12, fontweight='bold')
ax_top.set_ylabel('Radial Position y (m)', fontsize=12, fontweight='bold')
ax_top.set_title('EHD Pump - Top View (Charge Transport)', fontsize=13, fontweight='bold')
ax_top.grid(True, alpha=0.3)
ax_top.set_facecolor('#f0f0f0')

# Draw cylinder boundaries
ax_top.plot([0, 0], [-cylinder_radius, cylinder_radius], 'r-', linewidth=4, label='Left Electrode (+V₀)')
ax_top.plot([cylinder_length, cylinder_length], [-cylinder_radius, cylinder_radius], 'b-', linewidth=4, label='Right Electrode (0V)')
ax_top.axhline(y=cylinder_radius, color='gray', linestyle='--', linewidth=1, alpha=0.5)
ax_top.axhline(y=-cylinder_radius, color='gray', linestyle='--', linewidth=1, alpha=0.5)

# Side view (cross-section)
ax_side = plt.subplot(1, 2, 2)
ax_side.set_xlim(-0.001, cylinder_length + 0.001)
ax_side.set_ylim(-1000, V_0 + 1000)
ax_side.set_xlabel('Position z (m)', fontsize=12, fontweight='bold')
ax_side.set_ylabel('Potential V (V)', fontsize=12, fontweight='bold')
ax_side.set_title('Potential Profile & Particle Energy', fontsize=13, fontweight='bold')
ax_side.grid(True, alpha=0.3)
ax_side.set_facecolor('#f0f0f0')

# Plot potential profile
ax_side.plot(z, V, 'b-', linewidth=2, label='Potential V(z)')
ax_side.fill_between(z, 0, V, alpha=0.2, color='blue')

# Initialize particles
particles_top, = ax_top.plot([], [], 'o', color='#FFD700', markersize=5, 
                              markeredgecolor='black', markeredgewidth=0.5, alpha=0.9)
particles_side, = ax_side.plot([], [], 'o', color='#FF4500', markersize=5, 
                                markeredgecolor='black', markeredgewidth=0.5, alpha=0.9)

# Add legends
ax_top.legend(loc='upper right', fontsize=10)
ax_side.legend(loc='upper right', fontsize=10)

# Time text
time_text = ax_top.text(0.02, 0.98, '', transform=ax_top.transAxes, 
                        fontsize=11, verticalalignment='top',
                        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.9))

def init_anim():
    particles_top.set_data([], [])
    particles_side.set_data([], [])
    time_text.set_text('')
    return particles_top, particles_side, time_text

def animate(frame):
    global particle_z, particle_r, particle_vz, particle_theta
    
    # Update particle positions
    dt = 1.0 / fps
    
    # Velocity depends on electric field strength
    # Add some randomness to velocity for realistic motion
    velocity_variation = 1 + 0.2 * np.sin(2 * np.pi * frame / (fps * 2))
    current_velocity = base_velocity * velocity_scale * velocity_variation
    
    particle_z += current_velocity * dt
    
    # Add slight radial oscillation (Brownian motion effect)
    particle_r += 0.00008 * np.sin(2 * np.pi * frame / fps + particle_theta) * dt
    particle_theta += 0.1 * dt
    
    # Reset particles that exit
    exited = particle_z > cylinder_length
    particle_z[exited] = 0
    particle_r[exited] = np.random.uniform(-cylinder_radius, cylinder_radius, np.sum(exited))
    particle_theta[exited] = np.random.uniform(0, 2*np.pi, np.sum(exited))
    
    # Keep particles within cylinder
    particle_r = np.clip(particle_r, -cylinder_radius * 0.95, cylinder_radius * 0.95)
    
    # Update plot data
    particles_top.set_data(particle_z, particle_r)
    
    # Calculate potential at each particle position
    particle_potentials = -(rho_0 * particle_z**2) / (2 * epsilon) + A * particle_z + B
    particles_side.set_data(particle_z, particle_potentials)
    
    # Update time
    current_time = frame / fps
    avg_velocity_display = base_velocity * velocity_scale * 1000
    time_text.set_text(f'Time: {current_time:.2f} s\nParticles: {n_particles}\nAvg velocity: {avg_velocity_display:.2f} mm/s\nE-field: {E_field/1e6:.2f} MV/m')
    
    return particles_top, particles_side, time_text

# Create animation
print("Creating animation object...")
anim = FuncAnimation(fig_anim, animate, init_func=init_anim, 
                     frames=n_frames, interval=1000/fps, blit=True)

# Try multiple save methods
save_success = False

# Method 1: Try FFmpeg first
try:
    from matplotlib.animation import FFMpegWriter
    writer = FFMpegWriter(fps=fps, bitrate=2000, codec='libx264')
    anim.save('ehd_pump_animation.mp4', writer=writer, dpi=120)
    print("✓ Animation saved as 'ehd_pump_animation.mp4' (FFmpeg)")
    save_success = True
except Exception as e:
    print(f"✗ FFmpeg method failed: {e}")

# Method 2: Try Pillow/GIF format (always works)
if not save_success:
    try:
        print("Trying Pillow writer (GIF format)...")
        writer = PillowWriter(fps=fps)
        anim.save('ehd_pump_animation.gif', writer=writer, dpi=100)
        print("✓ Animation saved as 'ehd_pump_animation.gif' (GIF format)")
        save_success = True
    except Exception as e:
        print(f"✗ Pillow method failed: {e}")

# Method 3: Save as individual frames
if not save_success:
    print("Saving as individual frames...")
    import os
    os.makedirs('animation_frames', exist_ok=True)
    for i in range(min(n_frames, 100)):  # Save first 100 frames
        animate(i)
        plt.savefig(f'animation_frames/frame_{i:04d}.png', dpi=100)
        if i % 20 == 0:
            print(f"  Saved frame {i}/{n_frames}")
    print("✓ Frames saved in 'animation_frames/' folder")
    print("  You can combine them into a video using online tools or ffmpeg")

print(f"\nAnimation details:")
print(f"  Duration: {duration} seconds")
print(f"  Frame rate: {fps} fps")
print(f"  Total frames: {n_frames}")
print(f"  Number of particles: {n_particles}")

# Show the animation
print("\nDisplaying animation preview...")
plt.show()

print("\n" + "=" * 60)
print("SIMULATION COMPLETE!")
print("=" * 60)
print("Generated files:")
print("  1. ehd_pump_analysis.png - Comprehensive analysis plots")
if save_success:
    print("  2. ehd_pump_animation.mp4 or .gif - Particle transport animation")
else:
    print("  2. animation_frames/ - Individual animation frames")
print("=" * 60)
print("\nTo install FFmpeg for MP4 support:")
print("  - Windows: Download from https://ffmpeg.org/download.html")
print("  - Mac: brew install ffmpeg")
print("  - Linux: sudo apt-get install ffmpeg")
print("=" * 60)